In [1]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'keras_core'

In [4]:
print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

TensorFlow version: 2.16.1
KerasNLP version: 0.12.0


In [4]:
import pandas as pd
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [4]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Sentence and word tokenization using nltk

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize #Select any type of tokenization from the list of tokenizations available as per requirements

In [9]:
sent_tokenize(df_train["text"][0])

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all']

In [11]:
print(sent_tokenize("Kya hua tera vaadaa. Wo kasam wo iraada. Bhulega dil jis din tumhe, wo din zindagi ka aakhiri din hoga."))
print(word_tokenize("Kya hua tera vaadaa. Wo kasam wo iraada. Bhulega dil jis din tumhe, wo din zindagi ka aakhiri din hoga."))

['Kya hua tera vaadaa.', 'Wo kasam wo iraada.', 'Bhulega dil jis din tumhe, wo din zindagi ka aakhiri din hoga.']
['Kya', 'hua', 'tera', 'vaadaa', '.', 'Wo', 'kasam', 'wo', 'iraada', '.', 'Bhulega', 'dil', 'jis', 'din', 'tumhe', ',', 'wo', 'din', 'zindagi', 'ka', 'aakhiri', 'din', 'hoga', '.']


Sentence and word tokenization using spacy

In [12]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import spacy

In [14]:
nlp = spacy.load("en_core_web_sm") # Create language object

doc = nlp(df_train["text"][0])

In [15]:
for sentence in doc.sents:
  print(sentence)
  for token in sentence:
    print(token)

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Our
Deeds
are
the
Reason
of
this
#
earthquake
May
ALLAH
Forgive
us
all


In [16]:
nlp = spacy.blank("en")
doc = nlp(df_train["text"][0])

In [17]:
token0 = doc[0]
print(type(token0))
token0

<class 'spacy.tokens.token.Token'>


Our

In [ ]:
dir(token0)   #lists all the methods of token class #understands context, currency, email id and a lot more.

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'ancestors',
 'check_flag',
 'children',
 'cluster',
 'conjuncts',
 'dep',
 'dep_',
 'doc',
 'ent_id',
 'ent_id_',
 'ent_iob',
 'ent_iob_',
 'ent_kb_id',
 'ent_kb_id_',
 'ent_type',
 'ent_type_',
 'get_extension',
 'has_dep',
 'has_extension',
 'has_head',
 'has_morph',
 'has_vector',
 'head',
 'i',
 'idx',
 'iob_strings',
 'is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_end',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang',
 'lang_',
 'le

In [20]:
#Examples
print("First token value is - ", token0.text)
print("Is token an alphabet? " ,token0.is_alpha)
print("Is token a number? ", token0.like_num)
tokenx = nlp('two')[0]
print("Second token value is - ", tokenx.text)
print("Is token an alphabet? ", tokenx.is_alpha)
print("Is token a number? ", tokenx.like_num) #it understands the true meaning with context

First token value is -  Our
Is token an alphabet?  True
Is token a number?  False
Second token value is -  two
Is token an alphabet?  True
Is token a number?  True


In [21]:
#We can define our own special cases
from spacy.symbols import ORTH
nlp.tokenizer.add_special_case("gimme", [{ORTH : "gim"}, {ORTH : "me"}])

doc = nlp("gimme a large piece of dark chocolate cake.")
tokens = [token.text for token in doc]
print(tokens)

['gim', 'me', 'a', 'large', 'piece', 'of', 'dark', 'chocolate', 'cake', '.']


In [25]:
#A doc with multiple lines
doc = nlp("chakk dhum dhum. chakk dhum dhum. chakk dhum dhum.")
for sent in doc.sents:
  print(sent)        #This gives error as our nlp pipeline is blank

ValueError: [E030] Sentence boundaries unset. You can add the 'sentencizer' component to the pipeline with: `nlp.add_pipe('sentencizer')`. Alternatively, add the dependency parser or sentence recognizer, or set sentence boundaries by setting `doc[i].is_sent_start`.

In [26]:
nlp.pipe_names  #Blank

[]

In [29]:
nlp.add_pipe("sentencizer")
nlp.pipe_names

['sentencizer']

In [30]:
doc = nlp("chakk dhum dhum. chakk dhum dhum. chakk dhum dhum.")
for sent in doc.sents:
  print(sent)

chakk dhum dhum.
chakk dhum dhum.
chakk dhum dhum.


In [31]:
#nlp = spacy.blank(name="en") # Adds a blank pipeline
#nlp = spacy.load("en_core_web_sm") # Loads a pre-trained pipeline

In [32]:
nlp = spacy.load("en_core_web_sm")
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7c24f27df3a0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7c24effd7ee0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7c24f06154d0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7c24eff5d580>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7c24f2e7af40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7c24f06152a0>)]

In [37]:
doc = nlp("Dr. Haathi ate samosas for 100$ . He then said he is still hungry.")
for token in doc:
  print(token.text, " | ", token.pos_, " | ", token.lemma_)

Dr.  |  PROPN  |  Dr.
Haathi  |  PROPN  |  Haathi
ate  |  VERB  |  eat
samosas  |  NOUN  |  samosa
for  |  ADP  |  for
100  |  NUM  |  100
$  |  NUM  |  $
.  |  PUNCT  |  .
He  |  PRON  |  he
then  |  ADV  |  then
said  |  VERB  |  say
he  |  PRON  |  he
is  |  AUX  |  be
still  |  ADV  |  still
hungry  |  ADJ  |  hungry
.  |  PUNCT  |  .


In [38]:
#NER
for ent in doc.ents:
  print(ent.text, " | ", ent.label_, " | ", spacy.explain(ent.label_))

Haathi  |  PERSON  |  People, including fictional
100$  |  MONEY  |  Monetary values, including unit


In [39]:
from spacy import displacy
displacy.render(doc, style="ent")

In [40]:
#To add only NER component in the pipeline
source_nlp = spacy.load("en_core_web_sm")
nlp = spacy.blank("en")
nlp.add_pipe("ner", source=source_nlp)
nlp.pipe_names

['ner']

Stemming and Lematization

In [2]:
#Spacy doesn't support stemming

import nltk
import spacy
from nltk.stem import PorterStemmer, SnowballStemmer

stemmer = PorterStemmer()

words = ["eating", "eats", "eat", "ate", "Adjustable", "rafting", "ability", "meeting"]
for word in words:
  print(word, " | ", stemmer.stem(word))

eating  |  eat
eats  |  eat
eat  |  eat
ate  |  ate
Adjustable  |  adjust
rafting  |  raft
ability  |  abil
meeting  |  meet


In [3]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("eating eats eat ate Adjustable rafting ability meeting")
for token in doc:
  print(token.text, " | ", token.lemma_)

eating  |  eat
eats  |  eat
eat  |  eat
ate  |  eat
Adjustable  |  adjustable
rafting  |  raft
ability  |  ability
meeting  |  meeting


In [4]:
doc = nlp("The quick brown foxes were jumping over the lazy dogs, but the leaping cat remained unmoved, watching them quietly.")
for token in doc:
  print(token.text, " | ", token.lemma_)

The  |  the
quick  |  quick
brown  |  brown
foxes  |  fox
were  |  be
jumping  |  jump
over  |  over
the  |  the
lazy  |  lazy
dogs  |  dog
,  |  ,
but  |  but
the  |  the
leaping  |  leap
cat  |  cat
remained  |  remain
unmoved  |  unmoved
,  |  ,
watching  |  watch
them  |  they
quietly  |  quietly
.  |  .


In [9]:
#Adding custom rules to the lematization model

ar = nlp.get_pipe ("attribute_ruler")
ar.add([[{"TEXT" : "Bro"}], [{"TEXT" : "Brah"}]], { "LEMMA" : "Brother" })
doc = nlp("Bro! Where are you going? Lets dance Brah!")
for token in doc:
  print(token.text, " | ", token.lemma_)

Bro  |  Brother
!  |  !
Where  |  where
are  |  be
you  |  you
going  |  go
?  |  ?
Lets  |  let
dance  |  dance
Brah  |  Brother
!  |  !


POS Tagging

In [11]:
nlp = spacy.load(name="en_core_web_sm")
doc = nlp("Elon Musk flew to Mars yesterday. He carried a lot of biryani with him.")
for token in doc:
  print(token.text, " | ", token.pos_, " | ", spacy.explain(token.pos_), " | ", token.tag_, " | ", spacy.explain(token.tag_))

Elon  |  PROPN  |  proper noun  |  NNP  |  noun, proper singular
Musk  |  PROPN  |  proper noun  |  NNP  |  noun, proper singular
flew  |  VERB  |  verb  |  VBD  |  verb, past tense
to  |  ADP  |  adposition  |  IN  |  conjunction, subordinating or preposition
Mars  |  PROPN  |  proper noun  |  NNP  |  noun, proper singular
yesterday  |  NOUN  |  noun  |  NN  |  noun, singular or mass
.  |  PUNCT  |  punctuation  |  .  |  punctuation mark, sentence closer
He  |  PRON  |  pronoun  |  PRP  |  pronoun, personal
carried  |  VERB  |  verb  |  VBD  |  verb, past tense
a  |  DET  |  determiner  |  DT  |  determiner
lot  |  NOUN  |  noun  |  NN  |  noun, singular or mass
of  |  ADP  |  adposition  |  IN  |  conjunction, subordinating or preposition
biryani  |  NOUN  |  noun  |  NN  |  noun, singular or mass
with  |  ADP  |  adposition  |  IN  |  conjunction, subordinating or preposition
him  |  PRON  |  pronoun  |  PRP  |  pronoun, personal
.  |  PUNCT  |  punctuation  |  .  |  punctuation mar

In [13]:
count = doc.count_by(spacy.attrs.POS)
print(count)
print(doc.vocab[96].text, doc.vocab[100].text, doc.vocab[85].text)

{96: 3, 100: 2, 85: 3, 92: 3, 97: 2, 95: 2, 90: 1}
PROPN VERB ADP


Named Entity Recognition - NER

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [4]:
doc = nlp("Elon Musk is going to acquire Twitter for $45 billion.")

for ent in doc.ents:
  print(ent.text, " | ", ent.label_, " | ", spacy.explain(ent.label_))

Elon Musk  |  PERSON  |  People, including fictional
Twitter  |  PRODUCT  |  Objects, vehicles, foods, etc. (not services)
$45 billion  |  MONEY  |  Monetary values, including unit


In [5]:
from spacy import displacy
displacy.render(doc, style="ent")

In [6]:
nlp.pipe_labels["ner"]

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [7]:
doc = nlp("Tesla is going to acquire Twitter for $45 billion.")
for ent in doc.ents:
  print(ent.text, " | ", ent.label_, " | ", spacy.explain(ent.label_))

Tesla  |  ORG  |  Companies, agencies, institutions, etc.
Twitter  |  PRODUCT  |  Objects, vehicles, foods, etc. (not services)
$45 billion  |  MONEY  |  Monetary values, including unit


In [8]:
# Span class is like a list of token class elements
#Customize the Entity Name for special cases
from spacy.tokens import Span

s1 = Span(doc, 0, 1, label="ORG")
s2 = Span(doc, 5, 6, label="ORG")
doc.set_ents([s1, s2], default="unmodified")

In [9]:
for ent in doc.ents:
  print(ent.text, " | ", ent.label_, " | ", spacy.explain(ent.label_))

Tesla  |  ORG  |  Companies, agencies, institutions, etc.
Twitter  |  ORG  |  Companies, agencies, institutions, etc.
$45 billion  |  MONEY  |  Monetary values, including unit


In [ ]:
#How to build my own NER
'''#Approach 1 : Simple Lookup
Maintain a database of all the entities - Name, Country, Location,.... and it will map any occurrence from there. This doesn't involve NLP.'''

'''#Approach 2 : Rule based NER
Eg. XYZ Inc is ORG, Regex based rules etc.
Ref - https://ner.pythonhumanities.com/02_01_spaCy_Entity_Ruler.html'''

'''#Approach 3 : Machine Learning - CRF-Conditional Random Fields , BERT'''


Stop Words

In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

STOP_WORDS

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

In [7]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Elon Musk is going to acquire Twitter for $45 billion. The other one is not other but your divine brother.")
for token in doc:
  if token.is_stop:
    print(token.text)

is
to
for
The
other
one
is
not
other
but
your


In [14]:
print([token.text for token in doc if token.is_stop])

['is', 'to', 'for', 'The', 'other', 'one', 'is', 'not', 'other', 'but', 'your']


In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
  doc = nlp(text)
  '''  tokens = []
  for token in doc:
    if token.is_stop:
      continue
    tokens.append(token.lemma_)
  return " ".join(tokens)'''
  clean_text = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
  return ' '.join(clean_text)

#preprocess(doc)

In [6]:
df_train['preprocessed_text'] = df_train['text'].apply(preprocess)

In [7]:
df_test['preprocessed_text'] = df_test['text'].apply(preprocess)

In [8]:
df_train.head()

,id,keyword,location,text,target,preprocessed_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed Reason earthquake ALLAH forgive
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near La Ronge Sask Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive wildfire evacuation orde..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo Ruby Alaska smoke wildfire pour...


Text Representation #1 Bag Of Words (BOW)

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train["text"], df_train["target"], test_size=0.2, random_state=42)
x_train_pp, x_test_pp, y_train_pp, y_test_pp = train_test_split(df_train["preprocessed_text"], df_train["target"], test_size=0.2, random_state=42)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_train_bow = vectorizer.fit_transform(x_train.values)
x_test_bow = vectorizer.transform(x_test.values)

In [7]:
x_train_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
vectorizer.get_feature_names_out()[1500:1550]

array(['alarms', 'alas', 'alaska', 'alaskan', 'alba', 'albany', 'albeit',
       'alberta', 'albertans', 'album', 'albvdnwjtg', 'alcohol',
       'alcoholandmetal', 'alcoholism', 'aldridge', 'alec', 'aleisstokes',
       'alekalicante', 'alert', 'alerts', 'alex', 'alexalltimelow',
       'alexandrian', 'alexbelloli', 'alexeivolkov1', 'alexhammerstone',
       'alexis_sanchez', 'alexjacobsonpfs', 'alexshipppp', 'alfapedia',
       'algae', 'algeria', 'algtcn8baf', 'alhaji', 'alien', 'aliens',
       'alifaditha', 'align', 'alil', 'alisonannyoung', 'alive',
       'aliveafter5', 'all', 'allah', 'allay', 'allday_cumshots',
       'allegations', 'alleged', 'allegedly', 'allegiance'], dtype=object)

In [9]:
dir(vectorizer) # gives names of all methods of vectorizer class

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_feature_names',
 '_check_n_features',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_sort_features',
 '_stop_words_id',
 '_validate_data',
 '_validate_ngram_range',
 '_validate_params',
 '_validate_vocabulary',
 '_warn_for_unused_params',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'bui

In [10]:
vectorizer.vocabulary_

{'courageous': 4224,
 'and': 1653,
 'honest': 7922,
 'analysis': 1643,
 'of': 11752,
 'need': 11286,
 'to': 16333,
 'use': 17031,
 'atomic': 2008,
 'bomb': 2792,
 'in': 8384,
 '1945': 212,
 'hiroshima70': 7830,
 'japanese': 8815,
 'military': 10700,
 'refused': 13565,
 'surrender': 15666,
 'https': 8026,
 'co': 3863,
 'vhmtytptgr': 17226,
 'zachzaidman': 18381,
 '670thescore': 773,
 'wld': 17804,
 'shame': 14605,
 'if': 8242,
 'that': 16064,
 'golf': 7178,
 'cart': 3372,
 'became': 2411,
 'engulfed': 5776,
 'flames': 6455,
 'boycottbears': 2866,
 'tell': 15965,
 'barackobama': 2277,
 'rescind': 13689,
 'medals': 10512,
 'honor': 7926,
 'given': 7093,
 'us': 17015,
 'soldiers': 15051,
 'at': 1986,
 'the': 16072,
 'massacre': 10405,
 'wounded': 17898,
 'knee': 9394,
 'sign': 14756,
 'now': 11553,
 'amp': 1630,
 'rt': 14063,
 'u4r8driuac': 16770,
 'worried': 17883,
 'about': 1204,
 'how': 7991,
 'ca': 3184,
 'drought': 5323,
 'might': 10679,
 'affect': 1365,
 'you': 18285,
 'extreme': 608

In [11]:
x_train_np = x_train_bow.toarray()
x_test_np = x_test_bow.toarray()

In [18]:
print(x_train_np[0].shape) #18566
#To get indices of items which are not 0 for the first row of data
print(np.where(x_train_np[0] != 0))
print(vectorizer.get_feature_names_out()[np.where(x_train_np[0] != 0)])

(18566,)
(array([  212,  1643,  1653,  2008,  2792,  3863,  4224,  7830,  7922,
        8026,  8384,  8815, 10700, 11286, 11752, 13565, 15666, 16333,
       17031, 17226]),)
['1945' 'analysis' 'and' 'atomic' 'bomb' 'co' 'courageous' 'hiroshima70'
 'honest' 'https' 'in' 'japanese' 'military' 'need' 'of' 'refused'
 'surrender' 'to' 'use' 'vhmtytptgr']


In [39]:
#Lets build machine learning model on top of this feature using Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = MultinomialNB()
clf.fit(x_train_np, y_train)
y_pred = clf.predict(x_test_np)
print(classification_report(y_test, y_pred))

'clf = MultinomialNB()\nclf.fit(x_train_np, y_train)\ny_pred = clf.predict(x_test_np)\nprint(classification_report(y_test, y_pred))'

In [21]:
tweets = ['Catastrophic tide hit at the costal region of Bay of Bengal, 12 injured.', 'Damn! Such a huge disaster Shukla is.', 'What a bash it was last night!']
tweets_bow = vectorizer.transform(tweets)
tweets_np = tweets_bow.toarray()
clf.predict(tweets_np)

array([1, 0, 0])

In [22]:
from sklearn.pipeline import Pipeline
#Pipeline created will take care of text representation using Count vectorizer, so we can use raw data to feed our model
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])
clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       874
           1       0.81      0.70      0.75       649

    accuracy                           0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



In [59]:
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])
clf.fit(x_train_pp, y_train_pp)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])

In [60]:
y_pred = clf.predict(x_test_pp)
print(classification_report(y_test_pp, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83       874
           1       0.78      0.74      0.76       649

    accuracy                           0.80      1523
   macro avg       0.79      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



In [36]:
tweets = df_test["preprocessed_text"]
preds = clf.predict(tweets)

In [37]:
print("Total number of tweets = ", len(df_test), " |  No. of tweets predicted as disaster tweets = ", len(np.where(preds == 1)[0]), " | No. of tweets predicted as non disaster = ", len(np.where(preds == 0)[0]))

Total number of tweets =  3263  |  No. of tweets predicted as disaster tweets =  1282  | No. of tweets predicted as non disaster =  1981


Text Representation #2 Bag of n-grams

In [44]:
corpus = ["Whole country is dunbstruck by the catastrophy in Bhubaneshwar.", "Lost home, my world got shaken as the earth was shaking. #NepalEarthquake", "Party at Abhisha's."]
corpus_processed = [preprocess(text) for text in corpus]
corpus_processed

['country dunbstruck catastrophy Bhubaneshwar',
 'lose home world get shake earth shake NepalEarthquake',
 'Party Abhisha']

In [45]:
v = CountVectorizer(ngram_range=(1,2))
v.fit(corpus_processed)
v.vocabulary_

{'country': 4,
 'dunbstruck': 6,
 'catastrophy': 2,
 'bhubaneshwar': 1,
 'country dunbstruck': 5,
 'dunbstruck catastrophy': 7,
 'catastrophy bhubaneshwar': 3,
 'lose': 14,
 'home': 12,
 'world': 22,
 'get': 10,
 'shake': 19,
 'earth': 8,
 'nepalearthquake': 16,
 'lose home': 15,
 'home world': 13,
 'world get': 23,
 'get shake': 11,
 'shake earth': 20,
 'earth shake': 9,
 'shake nepalearthquake': 21,
 'party': 17,
 'abhisha': 0,
 'party abhisha': 18}

In [46]:
v.transform(['Whole country is dunbstruck by the catastrophy in Bhubaneshwar.']).toarray()

array([[0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [48]:
v.transform(['Party with Bhootnath.']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0]])

In [49]:
df_train['target'].value_counts()
#Handle class imbalance if required

target
0    4342
1    3271
Name: count, dtype: int64

In [42]:
vectorizer.vocabulary_

{'courageous and': 14892,
 'and honest': 3385,
 'honest analysis': 25170,
 'analysis of': 3089,
 'of need': 35916,
 'need to': 34133,
 'to use': 50229,
 'use atomic': 52059,
 'atomic bomb': 5094,
 'bomb in': 7382,
 'in 1945': 26088,
 '1945 hiroshima70': 424,
 'hiroshima70 japanese': 24878,
 'japanese military': 28316,
 'military refused': 32538,
 'refused surrender': 40658,
 'surrender https': 45932,
 'https co': 25523,
 'co vhmtytptgr': 13523,
 'zachzaidman 670thescore': 56595,
 '670thescore wld': 1079,
 'wld shame': 55128,
 'shame if': 43095,
 'if that': 25879,
 'that golf': 46853,
 'golf cart': 22582,
 'cart became': 9263,
 'became engulfed': 6221,
 'engulfed in': 18450,
 'in flames': 26309,
 'flames boycottbears': 20339,
 'tell barackobama': 46465,
 'barackobama to': 5738,
 'to rescind': 50094,
 'rescind medals': 40934,
 'medals of': 32102,
 'of honor': 35784,
 'honor given': 25182,
 'given to': 22361,
 'to us': 50228,
 'us soldiers': 52015,
 'soldiers at': 44307,
 'at the': 5046,


In [55]:
#Training on raw text
clf = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('classifier', MultinomialNB())
])
clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('classifier', MultinomialNB())])

In [56]:
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       874
           1       0.80      0.70      0.74       649

    accuracy                           0.80      1523
   macro avg       0.80      0.78      0.79      1523
weighted avg       0.80      0.80      0.79      1523



In [57]:
#Training on preprocessed text
clf = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('classifier', MultinomialNB())
])
clf.fit(x_train_pp, y_train_pp)

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('classifier', MultinomialNB())])

In [58]:
y_pred = clf.predict(x_test_pp)
print(classification_report(y_test_pp, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       874
           1       0.76      0.74      0.75       649

    accuracy                           0.79      1523
   macro avg       0.79      0.79      0.79      1523
weighted avg       0.79      0.79      0.79      1523



Text Representation #3 TF-IDF (Term frequency - Inverse Document Frequency)

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ["Whole country is dunbstruck by the catastrophy in Bhubaneshwar.",
          "Lost home, my world got shaken as the earth was shaking.",
          "#NepalEarthquake",
          "Party at Abhisha's",
          "Party with Bhootnath.",
          "The sun is burning and the land is shaking"]

In [3]:
vectorizer = TfidfVectorizer()
transformed_output = vectorizer.fit_transform(corpus)
print(vectorizer.vocabulary_)

{'whole': 26, 'country': 9, 'is': 15, 'dunbstruck': 10, 'by': 7, 'the': 24, 'catastrophy': 8, 'in': 14, 'bhubaneshwar': 5, 'lost': 17, 'home': 13, 'my': 18, 'world': 28, 'got': 12, 'shaken': 21, 'as': 2, 'earth': 11, 'was': 25, 'shaking': 22, 'nepalearthquake': 19, 'party': 20, 'at': 3, 'abhisha': 0, 'with': 27, 'bhootnath': 4, 'sun': 23, 'burning': 6, 'and': 1, 'land': 16}


In [4]:
dir(vectorizer)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_feature_names',
 '_check_n_features',
 '_check_params',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_sort_features',
 '_stop_words_id',
 '_tfidf',
 '_validate_data',
 '_validate_ngram_range',
 '_validate_params',
 '_validate_vocabulary',
 '_warn_for_unused_params',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',

In [6]:
all_features = vectorizer.get_feature_names_out()

for word in all_features:
  indx = vectorizer.vocabulary_.get(word)
  print(word, " | ",vectorizer.idf_[indx], " | ", transformed_output[0, indx])

abhisha  |  2.252762968495368  |  0.0
and  |  2.252762968495368  |  0.0
as  |  2.252762968495368  |  0.0
at  |  2.252762968495368  |  0.0
bhootnath  |  2.252762968495368  |  0.0
bhubaneshwar  |  2.252762968495368  |  0.3502477663942352
burning  |  2.252762968495368  |  0.0
by  |  2.252762968495368  |  0.3502477663942352
catastrophy  |  2.252762968495368  |  0.3502477663942352
country  |  2.252762968495368  |  0.3502477663942352
dunbstruck  |  2.252762968495368  |  0.3502477663942352
earth  |  2.252762968495368  |  0.0
got  |  2.252762968495368  |  0.0
home  |  2.252762968495368  |  0.0
in  |  2.252762968495368  |  0.3502477663942352
is  |  1.8472978603872037  |  0.28720817880702754
land  |  2.252762968495368  |  0.0
lost  |  2.252762968495368  |  0.0
my  |  2.252762968495368  |  0.0
nepalearthquake  |  2.252762968495368  |  0.0
party  |  1.8472978603872037  |  0.0
shaken  |  2.252762968495368  |  0.0
shaking  |  1.8472978603872037  |  0.0
sun  |  2.252762968495368  |  0.0
the  |  1.559

In [7]:
transformed_output.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.35024777, 0.        , 0.35024777, 0.35024777, 0.35024777,
        0.35024777, 0.        , 0.        , 0.        , 0.35024777,
        0.28720818, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.24248088,
        0.        , 0.35024777, 0.        , 0.        ],
       [0.        , 0.        , 0.31385582, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.31385582, 0.31385582, 0.31385582, 0.        ,
        0.        , 0.        , 0.31385582, 0.31385582, 0.        ,
        0.        , 0.31385582, 0.25736626, 0.        , 0.21728628,
        0.31385582, 0.        , 0.        , 0.31385582],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
  

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', KNeighborsClassifier())
])

In [19]:
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.86      0.82       874
           1       0.78      0.66      0.72       649

    accuracy                           0.78      1523
   macro avg       0.78      0.76      0.77      1523
weighted avg       0.78      0.78      0.77      1523



In [20]:
print(x_test[:5], y_test[:5], y_pred[0:5])

2644    So you have a new weapon that can cause un-ima...
2227    The f$&amp;@ing things I do for #GISHWHES Just...
5448    DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...
132     Aftershock back to school kick off was great. ...
6845    in response to trauma Children of Addicts deve...
Name: text, dtype: object 2644    1
2227    0
5448    1
132     0
6845    0
Name: target, dtype: int64 [0 0 0 0 1]


In [22]:
from sklearn.naive_bayes import MultinomialNB
clf2 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

clf2.fit(x_train, y_train)

y_pred = clf2.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       874
           1       0.86      0.63      0.73       649

    accuracy                           0.80      1523
   macro avg       0.82      0.78      0.78      1523
weighted avg       0.81      0.80      0.79      1523



In [23]:
print(x_test[:5], y_test[:5], y_pred[0:5])


2644    So you have a new weapon that can cause un-ima...
2227    The f$&amp;@ing things I do for #GISHWHES Just...
5448    DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...
132     Aftershock back to school kick off was great. ...
6845    in response to trauma Children of Addicts deve...
Name: text, dtype: object 2644    1
2227    0
5448    1
132     0
6845    0
Name: target, dtype: int64 [0 0 0 0 0]


In [24]:
from sklearn.ensemble import RandomForestClassifier

clf3 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', RandomForestClassifier())
])

clf3.fit(x_train, y_train)

y_pred = clf3.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83       874
           1       0.85      0.59      0.69       649

    accuracy                           0.78      1523
   macro avg       0.80      0.75      0.76      1523
weighted avg       0.79      0.78      0.77      1523



In [25]:
print(x_test[:5], y_test[:5], y_pred[0:5])

2644    So you have a new weapon that can cause un-ima...
2227    The f$&amp;@ing things I do for #GISHWHES Just...
5448    DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...
132     Aftershock back to school kick off was great. ...
6845    in response to trauma Children of Addicts deve...
Name: text, dtype: object 2644    1
2227    0
5448    1
132     0
6845    0
Name: target, dtype: int64 [0 0 1 0 0]


Text Representation #4 Word Embeddings

Word Vector in spacy

In [9]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [13]:
doc = nlp("dog cat banana avocado tukonbe")
for token in doc:
  print(token.text, " | ", token.has_vector, " | ", token.vector_norm, " | ", token.is_oov)

dog  |  True  |  75.254234  |  False
cat  |  True  |  63.188496  |  False
banana  |  True  |  31.620354  |  False
avocado  |  True  |  28.434193  |  False
tukonbe  |  False  |  0.0  |  True


In [14]:
doc[0].vector.shape

(300,)

In [15]:
base_token = nlp("bread")
doc = nlp("avocado sandwich burger bun tiger car puppy")
for token in doc:
  print(base_token.text, " | ", token.text, " | ", token.similarity(base_token))

bread  |  avocado  |  0.5338742473085764
bread  |  sandwich  |  0.6341067010130894
bread  |  burger  |  0.47520687769584247
bread  |  bun  |  0.5145848627438849
bread  |  tiger  |  0.04764611675903374
bread  |  car  |  0.06451533308853552
bread  |  puppy  |  0.14947537432215535


In [16]:
def print_similarity(base_token, doc_to_compare):
  base_token = nlp(base_token)
  doc = nlp(doc_to_compare)
  for token in doc:
    print(base_token.text, " | ", token.text, " | ", token.similarity(base_token))

In [17]:
print_similarity("IPhone", "mobile phone samsung delivery eat dance technology")

IPhone  |  mobile  |  0.5727632820959091
IPhone  |  phone  |  0.7233242169713832
IPhone  |  samsung  |  0.5804839618803266
IPhone  |  delivery  |  0.219182460545024
IPhone  |  eat  |  -0.1273598518796623
IPhone  |  dance  |  0.0014647329390619478
IPhone  |  technology  |  0.26274597466629546


In [20]:
king = nlp.vocab["king"].vector
queen = nlp.vocab["queen"].vector
man = nlp.vocab["man"].vector
woman = nlp.vocab["woman"].vector

result = king - man + woman

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([result], [queen])

array([[0.61780137]], dtype=float32)

In [10]:
df_train.target.value_counts() #Check for class imbalance

target
0    4342
1    3271
Name: count, dtype: int64

In [13]:
import spacy

nlp = spacy.load("en_core_web_lg")
doc = nlp(df_train.text[0])
doc.vector.shape

(300,)

In [11]:
df_train['vector'] = df_train['text'].apply(lambda x: nlp(x).vector)

In [12]:
df_train.head()

,id,keyword,location,text,target,preprocessed_text,vector
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed Reason earthquake ALLAH forgive,"[-0.965633, 0.086769275, -2.2507236, 0.4783793..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near La Ronge Sask Canada,"[-1.8681643, -1.6165241, -0.21609005, -1.76522..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...,"[-2.4070368, 0.27042598, -1.3639991, 0.7013667..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive wildfire evacuation orde...","[-1.0255101, -0.15121445, -2.2331533, 1.724021..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo Ruby Alaska smoke wildfire pour...,"[0.08517767, -0.10672835, -1.9185519, 0.144816..."


In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train["vector"].values, df_train["target"], test_size=0.2, random_state=2022)

In [14]:
import numpy as np

x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

scalar = MinMaxScaler()
x_train_2d_scaled = scalar.fit_transform(x_train_2d)
x_test_2d_scaled = scalar.transform(x_test_2d)

clf = MultinomialNB()
clf.fit(x_train_2d_scaled, y_train)

MultinomialNB()

In [24]:
y_pred = clf.predict(x_test_2d_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.70      0.71       862
           1       0.63      0.66      0.65       661

    accuracy                           0.68      1523
   macro avg       0.68      0.68      0.68      1523
weighted avg       0.69      0.68      0.68      1523



In [26]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')
clf.fit(x_train_2d_scaled, y_train)

y_pred = clf.predict(x_test_2d_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.82      0.79       862
           1       0.74      0.66      0.70       661

    accuracy                           0.75      1523
   macro avg       0.75      0.74      0.75      1523
weighted avg       0.75      0.75      0.75      1523



Word Vectors in Gensim overview

In [16]:
import gensim.downloader as api

wv = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


In [17]:
wv.similarity(w1="disaster", w2="catastrophe")

0.6707797

In [18]:
wv.similarity(w1="well", w2="great")

0.8710875

In [26]:
wv_great = wv["great"]
wv_well = wv["well"]
print(wv_great.shape, wv_well.shape)

#Sentence embedding is the mathematical embedding of all the word embeddings in the sentence, and is get by wv.get_mean_vector
import numpy as np
print(np.mean([wv_great, wv_well], axis=0)[:3])
print(wv.get_mean_vector(["great", "well"], pre_normalize=False)[:3])

(50,) (50,)
[0.43752003 0.14395651 0.322215  ]
[0.43752003 0.14395651 0.322215  ]


In [19]:
wv.most_similar("disaster")

[('tragedy', 0.8369920253753662),
 ('destruction', 0.79105144739151),
 ('relief', 0.7765377163887024),
 ('hurricane', 0.7673801183700562),
 ('apocalypse', 0.7665935158729553),
 ('climate', 0.7536067366600037),
 ('operation', 0.7534821629524231),
 ('threat', 0.7521082758903503),
 ('caused', 0.7516399025917053),
 ('death', 0.7499884366989136)]

In [32]:
wv.most_similar(positive=["france", "berlin"], negative=["paris"])

[('deutschland', 0.8015620708465576),
 ('frankfurt', 0.7949166297912598),
 ('hamburg', 0.7809246182441711),
 ('hannover', 0.7617123126983643),
 ('germany', 0.74187171459198),
 ('düsseldorf', 0.7412071824073792),
 ('ukraine', 0.7311040759086609),
 ('münchen', 0.7288892269134521),
 ('wien', 0.7238292098045349),
 ('deutsche', 0.7222028374671936)]

In [33]:
wv.most_similar(positive=["king", "woman"], negative=["man"])

[('prince', 0.759779691696167),
 ('stone', 0.7595877051353455),
 ('queen', 0.754626989364624),
 ('meets', 0.7404001355171204),
 ('royal', 0.7376410365104675),
 ('african', 0.7345032691955566),
 ('american', 0.7139527201652527),
 ('virgin', 0.7101486325263977),
 ('mother', 0.702320396900177),
 ('called', 0.6990438103675842)]

In [34]:
wv.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [27]:
df_train.head()

,id,keyword,location,text,target,preprocessed_text,vector
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed Reason earthquake ALLAH forgive,"[-0.965633, 0.086769275, -2.2507236, 0.4783793..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near La Ronge Sask Canada,"[-1.8681643, -1.6165241, -0.21609005, -1.76522..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...,"[-2.4070368, 0.27042598, -1.3639991, 0.7013667..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive wildfire evacuation orde...","[-1.0255101, -0.15121445, -2.2331533, 1.724021..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo Ruby Alaska smoke wildfire pour...,"[0.08517767, -0.10672835, -1.9185519, 0.144816..."


In [38]:
#preprocess the text through en_core_web_lg so that it includes word embeddings
nlp = spacy.load("en_core_web_lg")
def preprocess_and_vectorize(text):
  filtered_tokens = preprocess(text)
  #print(filtered_tokens)
  return wv.get_mean_vector(filtered_tokens.split(' ')) #used .split since we need the list with each word as a string element

In [35]:
preprocess_and_vectorize(df_train.text[0])

deed reason earthquake ALLAH forgive


array([ 6.52188659e-02,  1.15682259e-01, -2.43483111e-03,  3.83654125e-02,
       -7.81096071e-02,  5.51525056e-02,  7.45704472e-02, -8.05256888e-02,
       -6.99309707e-02,  2.48673931e-02,  3.26804537e-03, -6.71479106e-02,
       -5.70622981e-01, -8.93258899e-02,  2.31612846e-03,  9.98653769e-02,
        4.58067209e-02,  7.06321970e-02,  3.90785933e-02, -7.53571689e-02,
        4.25346605e-02,  6.77691549e-02,  8.78870487e-05,  8.78601372e-02,
        1.08209208e-01,  1.21982954e-01,  3.18823010e-03,  1.17184483e-02,
        3.95699926e-02,  3.99554744e-02, -1.65850669e-02,  7.42356852e-03,
        7.67733753e-02, -5.17295338e-02,  2.21132487e-02,  6.59877062e-02,
       -1.06312037e-01, -2.58806460e-02,  1.04271630e-02, -5.48203476e-02,
        8.24308395e-03,  5.72562218e-04,  8.64937901e-02, -4.65951711e-02,
       -5.55786863e-02, -6.66726902e-02,  3.12515944e-02, -1.26918405e-03,
        4.64003570e-02,  1.51862249e-01], dtype=float32)

In [39]:
df_train['gensim_vector'] = df_train['text'].apply(lambda text: preprocess_and_vectorize(text))

In [40]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train["gensim_vector"].values, df_train["target"], test_size=0.2, random_state=2022, stratify=df_train["target"])

In [42]:
x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [44]:
x_train_2d[:2]#flattened sample

array([[-3.87765393e-02,  8.47177207e-02, -5.11571355e-02,
         4.42314260e-02, -5.37176803e-02, -3.96499820e-02,
        -5.52360788e-02, -2.31958814e-02,  8.18864722e-03,
        -1.67825669e-01,  1.53111639e-02,  5.00255227e-02,
        -2.88256347e-01,  4.22041118e-02,  1.34411722e-01,
         6.31340127e-03,  1.95900258e-02,  4.80907373e-02,
         6.10472076e-02, -3.06147034e-03, -4.66695726e-02,
         3.71669754e-02, -6.08766787e-02, -9.94098857e-02,
        -1.14903495e-01,  1.43331602e-01,  6.02940395e-02,
         4.98709094e-04,  8.40000995e-03,  5.76666594e-02,
         1.04669556e-01, -1.08053433e-02,  4.91992980e-02,
        -1.24031916e-01,  5.23034632e-02, -5.11209145e-02,
         3.19463201e-02, -5.39161265e-04,  2.16923952e-02,
        -7.36937970e-02,  2.95985881e-02,  8.02823901e-02,
         4.60351035e-02, -2.09665429e-02,  5.41024515e-03,
         5.87697849e-02, -3.07617448e-02, -6.61654323e-02,
        -7.52645719e-04,  1.00365862e-01],
       [-1.07

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf = GradientBoostingClassifier()
clf.fit(x_train_2d, y_train)

y_pred = clf.predict(x_test_2d)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.84      0.80       869
           1       0.76      0.67      0.71       654

    accuracy                           0.77      1523
   macro avg       0.76      0.76      0.76      1523
weighted avg       0.77      0.77      0.76      1523



FastText